<a href="https://colab.research.google.com/github/roldugina/data_analysis_projects/blob/main/Housing_price_Canada.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 1. Loading the dataset and importing libraries

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [3]:
df=pd.read_csv("https://raw.githubusercontent.com/roldugina/data_analysis_projects/main/Homes%20for%20Sale%20and%20Real%20Estate.csv")

## 2. General dataset overview. Data cleaning (checking for missing values and anomalies)

In [4]:
df.shape

(3360, 8)

In [7]:
df.head()

,Address,Price,Description,Place,Beds,Bath,Sq.Ft,Website
0,3704 42 St SW,979999,CA AB T3E 3N1,Glenbrook,4,3.5,1813,Century 21 Bravo Realty
1,30 Mahogany Mews SE #415,439900,CA AB T3M 3H4,Mahogany,2,2.0,1029,Century 21 Bamber Realty Ltd.
2,273 Auburn Shores Way SE,950000,CA AB T3M 2E9,Auburn Bay,4,2.5,2545,Exp Realty
3,235 15 Ave SW #404,280000,CA AB T2R 0P6,Beltline,2,2.0,898,RE/MAX Realty Professionals
4,24 Hemlock Crescent SW #2308,649000,CA AB T3C 2Z1,Spruce Cliff,2,2.0,1482,Charles


In [6]:
df.describe()

,Price,Beds,Bath,Sq.Ft
count,3.360000e+03,3360.000000,3360.000000,3360.000000
mean,7.500648e+05,3.125000,2.479167,1558.815179
std,6.447619e+05,1.349087,1.023546,1056.424307
min,1.419000e+05,1.000000,1.000000,307.000000
25%,3.990000e+05,2.000000,2.000000,933.000000
50%,6.199000e+05,3.000000,2.500000,1352.000000
75%,8.499220e+05,4.000000,3.500000,2023.250000
max,1.000000e+07,10.000000,8.000000,39654.000000


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3360 entries, 0 to 3359
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Address      3360 non-null   object 
 1   Price        3360 non-null   int64  
 2   Description  3360 non-null   object 
 3   Place        3344 non-null   object 
 4   Beds         3360 non-null   int64  
 5   Bath         3360 non-null   float64
 6   Sq.Ft        3360 non-null   int64  
 7   Website      3359 non-null   object 
dtypes: float64(1), int64(3), object(4)
memory usage: 210.1+ KB


As far as we can see from Info, there are 2 columns with missing values: Place and Website. These variables are categorical and we cannot fill them with any values, so we'll drop these rows.

Checking variables types.
It is correct that all categorical columns (Address, Description, Place, Website) have object type. Price, Beds, Sq.Ft have integer type.

Bath column has float type, and we have to check this column in further analysis.


## 3. Cleaning the dataset

In [17]:
df['Bath'].unique()

array([3.5, 2. , 2.5, 1. , 3. , 1.5, 4.5, 5. , 6.5, 4. , 5.5, 6. , 7.5,
       8. ])

**Cleaning null values** (rows with null Place and Website columns).

In [9]:
df_cleaned=df.dropna()

In [10]:
len(df_cleaned)-len(df)

-17

**Cleaning not null values**

Checking for duplicates

In [11]:
df['Address'].duplicated().any()

False

There are no duplicated addresses

Checking Bath column

In [18]:
df['Bath'].unique()

array([3.5, 2. , 2.5, 1. , 3. , 1.5, 4.5, 5. , 6.5, 4. , 5.5, 6. , 7.5,
       8. ])

In [12]:
price_upper_limit=df['Price usd per square'].mean()+3*df['Price usd per square'].std()
price_lower_limit=df['Price usd per square'].mean()-2*df['Price usd per square'].std()

KeyError: 'Price usd per square'

In [ ]:
mask_price=df['Price usd per square']>price_upper_limit

In [ ]:
df_cleaned.loc[mask_price]

In [ ]:
mask_beds=df_cleaned['Beds']==1

In [ ]:
df['Sq.M']=df['Sq.Ft']/10.764
df['Price usd']=df['Price']/1.33
df['Price usd per square']=df['Price usd']/df['Sq.M']
df['Avg beds']=df['Sq.M']/df['Beds']

In [ ]:
df_cleaned[mask_beds]

In [ ]:
df_cleaned['Price usd per square'].sort_values(ascending=False)

In [ ]:
df_cleaned['Avg beds'].sort_values(ascending=False)

In [ ]:
df['Sq.Ft'].plot(figsize=[16,9])

In [ ]:
mask_max_beds=df_cleaned['Avg beds']==df_cleaned['Avg beds'].max()

In [ ]:
df_cleaned.loc[mask_max_beds]

In [ ]:
df_cleaned.loc[1584,'Sq.Ft']=df_cleaned.loc[1584,'Sq.Ft']=3684

In [ ]:
df_cleaned.describe()

In [ ]:
df_cleaned['Price usd']=df_cleaned['Price']/1.33
df_cleaned['Avg beds']=df_cleaned['Sq.M']/df_cleaned['Beds']
df_cleaned['Sq.M']=df_cleaned['Sq.Ft']/10.764
df_cleaned['Price usd per square']=df_cleaned['Price usd']/df_cleaned['Sq.M']

In [ ]:
plot_objects = plt.subplots(nrows=2, ncols=2, figsize=(24, 10))
fig,((ax1,ax2),(ax3,ax4))=plot_objects

In [ ]:
ax1.plot(df_cleaned['Price'],color='blue',linewidth=1,label='Price')
ax2.plot(df_cleaned['Sq.Ft'],color='green',linewidth=1)
ax3.hist(df_cleaned['Avg beds'],bins=100)
ax4.hist(df_cleaned['Beds'],bins=10)


ax1.set_ylabel('Price')
ax2.set_ylabel('Square Ft')

ax1.set_title("Price")
ax2.set_title("Square Ft")
ax1.legend()

fig.suptitle('House prices in Calgary 2023',fontsize=30)

fig